In [3]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

In [16]:
model=ChatOllama(model='llama3.1:8b')

In [6]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-08-07T11:01:53.654551742Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 12555726731, 'load_duration': 20470234, 'prompt_eval_count': 16, 'prompt_eval_duration': 2506952000, 'eval_count': 26, 'eval_duration': 9985072000}, id='run-dd2178df-a546-4f2c-99c8-ddb092bda97e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42})

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm a large language model, I don't have any information about your personal identity, including your name. This conversation just started, and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to know it!", response_metadata={'model': 'llama3.1:8b', 'created_at': '2024-08-07T11:02:18.900151556Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 25217638537, 'load_duration': 29963882, 'prompt_eval_count': 16, 'prompt_eval_duration': 2633182000, 'eval_count': 58, 'eval_duration': 22553390000}, id='run-8c4c2fb6-4648-4d8f-9b5e-18924bef6969-0', usage_metadata={'input_tokens': 16, 'output_tokens': 58, 'total_tokens': 74})

# Basic

In [5]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Nice to meet you, Bob! Is there something I can help you with or would you like to chat?'

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model


response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Nice to meet you, Bob! How's your day going so far? Is there anything I can help you with or would you just like to chat?"

# Document Loader

In [15]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "/home/bikasherl/Desktop/Week 10/LEAVE POLICY BAJRA TECHNOLOGIES 2081_82.docx.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load()


# print(pages[0].page_content)

# RAG

In [18]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings





text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


response = rag_chain.invoke({"input": "What is this document about? Please answer in one sentence."})
response["answer"]

/home/bikasherl/miniconda3/envs/bajra/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


'This document appears to be a human resources policy outlining guidelines for employee leaves, including annual leave, sick leave, and emergency situations.'